In [1]:
import sculib
from cam_sensors import sensor_data_pvsn
#moved imports separate to see if all modules available

import datetime
import numpy as np

# following to track sources
import katpoint
from astropy.time import Time
from astropy import units as u  
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
#from katpoint.refraction import refraction_offset_vlbi

#some global constants for MPI?
mpi_lat='-30:43:04.7'
mpi_lon='21:24:46.9'
mpi_alt=1086.0


In [2]:
sim = sculib.scu()
#sim.ip = '10.96.254.76'
#mpi = sculib.scu()
#mpi.ip = '10.96.64.10'


In [3]:
def ska_unwrap_tt(t, az, el):
    #Basic unwrap azimuth to be negative if az angle larger than 270
    uaz = np.array(list(map(lambda x: x-360 if x > 270 else x, az )))
    '''
    :#Test code
    az = range(0,361,10)
    unwrap_az = map(lambda x: x-360 if x > 270 else x, az )
    '''
    return(t, uaz, el)

def add_refract_tt(t, az, el, r_temp, r_press, r_rh, o_lat, o_lon, o_alt):
    '''
    The refraction model is based on that implemented in ERFA, which is fast but becomes inaccurate for altitudes 
    below about 5 degrees. Near and below altitudes of 0, it can even give meaningless answers, and in this case 
    transforming to AltAz and back to another frame can give highly discrepant results. For much better numerical 
    stability, leaving the pressure at 0 (the default), disabling the refraction correction (yielding 
    “topocentric” horizontal coordinates).
    
    altitude for elevation and height for altitude, really?
    '''
    #constant  
    #obswl is observation wavelength, optical 300 to 700nm
    owl = 0.00000055*u.m  #owl*u.m
    #astropy need to know the units
    temp = r_temp*u.Celsius
    presr = r_press/1000*u.bar
    RH=r_rh/100    #humidity is relative to 1
    #user in ERFA CRD
    loc = EarthLocation.from_geodetic(lat=o_lat, lon=o_lon, height=o_alt*u.m, ellipsoid='WGS84')
    T0 = Time(t[0], format='mjd') # Irrelevant, but necessary for astropy transformations
    
    #mother function, has scaling and T0
    erfa_crd = lambda ele, sc, presr, temp, RH : AltAz(alt=ele*u.deg, az=0*u.deg, location=loc, obstime=T0).transform_to( AltAz(obstime=T0, location=loc, obswl=owl, pressure=sc*presr, temperature=sc*temp, relative_humidity=sc*RH))
    #calls mother with two sc of 1 and 0 the convert to degrees?    
    erfa = lambda ele, presr, temp, RH: (erfa_crd(ele,1,presr,temp,RH).alt - erfa_crd(ele,0,presr,temp,RH).alt).deg # deg to add to unrefracted altitude

    refco = erfa(el, presr, temp, RH) # deg
    rel = el + refco
    
    return(t, az, rel) 

def gen_target_tt(observer, target, UTC_start, duration, dt=0.1, verbose=False, fname=None):
    """
        Convert J2000 RA,DEC to topocentric Az,El angles at time intervals as specified.
        Note: Topocentric angle is the apparent position computed for an observer at a specific location 
        on the surface of the earth, including corrections for geocentric parallax, diurnal abberation and 
        Earth rotation angle but neglecting atmospheric refraction.
        
        Example:
            R2D = 180/np.pi
            SDQM = katpoint.Antenna("SDQM", latitude=-30.7172/R2D, longitude=21.4129/R2D, altitude=1053.0) 
            # TODO: lat & lon
            gen_target_tt(SDQM, ra = "05:34:30", dec = "22:00:57", UTC_start = datetime.datetime(2019,10,14,8,0,0), 
                            duration = 300, dt = 0.1, verbose = True)
            
        @param observer: [katpoint.Antenna]
        @param target: either a string (then tgt_name) or tuple (then ra,dec) as below
        @param tgt_name: target name for SIMBAR query [string]
        @param ra : [string or float] Right ascension, either in 'H:M:S' or decimal degree string format, 
                     or as a float in radians.
        @param dec : [string or float] Declination, either in 'D:M:S' or decimal degree string format,
                     or as a float in radians.
        @param UTC_start: now as datetime time of first coordinate in UTC
        @param duration: [float] duration of the track table, now in seconds not hours ppak 1/7/2021.
        @param dt: [float] interval between coordinates in the track table, in seconds.
        @return: [list of tuples: (time, Az, El)] with time no in MJD ; 
                 topocentric Az in degrees East of North; topocentric El in degrees above the horizon.
    """
    #Conversion Radian to Degrees
    R2D = 180/np.pi  
    
    #Astropy SkyCoord thingy
    tgt = SkyCoord.from_name(target).icrs
    ra, dec = tgt.ra.rad, tgt.dec.rad 
    #Used Astropy to get ra and dec to feed into katpoint to create a katpoint target
    target = katpoint.construct_radec_target(ra, dec)

    #Get a start time from the function to create unix timestamp for T0
    #add so many seconds of observation to it in timestamps of dt
    #was  T0 = datetime.datetime(*UTC_start, tzinfo=datetime.timezone.utc).timestamp()
    T0 = UTC_start.timestamp()
    T = T0 + np.arange(0, duration, dt)
    
    #katpoint target get azel by feeding it unix time stamps and an observer!
    taz, tel = target.azel(T, observer) # radians
    #convert radians in az/el to degrees for antenna
    taz = taz*R2D
    tel = tel*R2D

    return(Time(T, format='unix').mjd, taz, tel)

def get_enviro(T0, duration):
    '''
    returns mean of temp, pres, humidity from T0-duration until T0
    
    
    T0: time now as datetime
    duration: how far back in past as seconds you want to get values of temp, pres, humidity
    '''
    #use receptor m000, it does not matter which one you use all receptors use the same temperature stations
    receptor = 'm000'
    #list of sensors used for reference, below we query them directly
    sensors=['_enviro_air_pressure',
             '_enviro_air_relative_humidity',
             '_enviro_air_temperature']

    #get time now, needed to request temp, press and relative humidity
    timestop=T0.timestamp()
    #time start for sensor retrieval as far back as we are going forward
    timestart=(T0 - datetime.timedelta(seconds=duration)).timestamp()

    #sensor_date_pvsn returns timestamp data as well, but not used
    timestampv,timestamps,air_temp=sensor_data_pvsn(receptor+'_enviro_air_temperature', timestart, timestop)
    mean_air_temp=np.mean(air_temp)

    timestampv,timestamps,air_press=sensor_data_pvsn(receptor+'_enviro_air_pressure', timestart, timestop)
    mean_air_press=np.mean(air_press)

    timestampv,timestamps,air_rh=sensor_data_pvsn(receptor+'_enviro_air_relative_humidity', timestart, timestop)
    mean_air_rh=np.mean(air_rh)

    return (mean_air_temp, mean_air_press, mean_air_rh)

In [4]:
def mpi_gen_tt(target, start, track_len, dt=0.1):
    '''
    target: a recognised name from astropy
    start:  a start time in datetime, default now in UTC
    track_len: how long to track source for in seconds
    observer hardcoded to be ska mpi from lat long defined in global constants.
    returns: track table, unwrapped and with refraction correction.
    '''
    # generate track tables
    observer_skampi = katpoint.Antenna("SKA_MPI", latitude=mpi_lat, longitude=mpi_lon, altitude=mpi_alt)
    T0 = start #
    
    print('Target: {} *** Start: {} *** Duration: {:.0f} seconds'.format(target, start, track_len))
    #raw track table as from katpoint
    raw_tt = gen_target_tt(observer_skampi, target, UTC_start=T0, duration=track_len, dt=dt)
    #unwrapped track table as for mpi dish
    unwrap_tt = ska_unwrap_tt(*raw_tt)
    #refraction correction as for current sensor values?
    temp, pres, rh = get_enviro(T0, track_len)
    print('Temp: {:.1f} Pres: {:.1f} RH: {:1f}'.format(temp, pres, rh))
    tt = add_refract_tt(*unwrap_tt, temp, pres, rh, mpi_lat, mpi_lon, mpi_alt)
    return(tt)

In [5]:
sampling = 200
    
def mpi_track_source(scu, target, start, track_len, dt):
    #Optical pointing tests 
    print('Test 316_000000_043')

    #Revisions:
    #0.1 initial write

    #Common setup
    config_name = 'HN_INDEX_TEST'
    wait5 = 5

    scu.debug = False
    
    if scu.logger_state() == 'RECORDING':
        print('WARNING, already recording - attempting to stop and start a fresh logger')
        scu.stop_logger()  
        scu.wait_duration(wait5)
    if scu.logger_state() == 'STOPPED':
        print('Starting logger with config: {}'.format(config_name))
        scu.start_logger(config_name)
        scu.wait_duration(wait5)
    tt=mpi_gen_tt(target, start, track_len, dt)
    #print(tt)
    scu.acu_ska_track(sim.format_body(*tt))
    scu.wait_duration(10)
    
def mpi_track_offset(scu, az_offset, el_offset):
    #ok so we have a running track, can we add some offset to it???
    scu.debug = False
    print('CURRENT OFFSET')
    print('Az: {} *** El: {} \n'.format(scu.status_Value("acu.tracking.act_stat_offset_value_az"), 
                                     scu.status_Value("acu.tracking.act_stat_offset_value_el")))
    scu.status_Value("acu.tracking.act_stat_offset_value_az")
    scu.status_Value("acu.tracking.act_stat_offset_value_el")
    
    scu.load_static_offset(az_offset, el_offset)
    scu.wait_duration(1)
    print('\n NEW OFFSET')
    print('Az: {} *** El: {}'.format(scu.status_Value("acu.tracking.act_stat_offset_value_az"), 
                                     scu.status_Value("acu.tracking.act_stat_offset_value_el")))

def mpi_save_tracklog(scu, target):
    wait5 = 5

    #This block required to stop, rename and save to disk the track logs
    #Step x
    scu.stop_logger()
    scu.wait_duration(wait5)
    print(scu.logger_state())

    #Step x
    print('Extract the data on the SCU @ 5Hz sampling')
    scu.save_session('316_000000_043_' + target , interval_ms = sampling)
    

In [6]:
targets=['Tania Australis',
         'theta1 Eri',
         'Tiaki']

target=targets[1]        

mpi_track_source(sim, target, start=datetime.datetime.now(datetime.timezone.utc), track_len=5*60, dt=1)


Test 316_000000_043
logger state 


ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /datalogging/currentState (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3fd2de6a90>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
#Example function to add offsets to az and/or el, it displays current offsets - makes a change - wait 1s - displays new offsets
mpi_track_offset(sim, 0, 0)

In [ ]:
Notebook needs to stop here, to allow track table to complete, before saving acu log to file 
by running the cell below manually when safe to do so.


In [ ]:
mpi_save_tracklog(sim, target)

In [ ]:
mpi_track_offset(sim, -140, -40)

In [ ]:
#Example function to add offsets to az and/or el, it displays current offsets - makes a change - wait 1s - displays new offsets
mpi_track_offset(sim, 0, 0)

All cells below just for debug


In [41]:
from astropy.coordinates import solar_system_ephemeris

from astropy.coordinates import get_body_barycentric, get_body, get_moon

T0 = datetime.datetime.now(datetime.timezone.utc).timestamp()
T = T0 + np.arange(0, 24*3600, 3600)
print(type(T0))

loc = EarthLocation.from_geodetic(lat=mpi_lat, lon=mpi_lon, height=mpi_alt*u.m, ellipsoid='WGS84')

with solar_system_ephemeris.set('builtin'):
    jup = get_body('jupiter', Time(T, format='unix', scale='utc'), loc) 
#jup  

target='PKS1934-63'
target='mars'
target=jup
#tt=mpi_gen_tt( target, start=datetime.datetime.now(datetime.timezone.utc), track_len=24*60*60, dt=60*60)


<class 'float'>


In [16]:
print(tt)

(array([59241.15326378, 59241.19493045, 59241.23659711, 59241.27826378,
       59241.31993045, 59241.36159711, 59241.40326378, 59241.44493045,
       59241.48659711, 59241.52826378, 59241.56993045, 59241.61159711,
       59241.65326378, 59241.69493045, 59241.73659711, 59241.77826378,
       59241.81993045, 59241.86159711, 59241.90326378, 59241.94493045,
       59241.98659711, 59242.02826378, 59242.06993045, 59242.11159711]), array([150.3464382 , 149.00426505, 149.46479738, 152.22950769,
       157.91296274, 166.89819954, 178.49741803, 190.43788644,
       200.18993491, 206.67088285, 210.09354528, 211.06612324,
       210.12822899, 207.66263998, 203.93338401, 199.14536676,
       193.49777021, 187.22265685, 180.60078845, 173.94968684,
       167.59024797, 161.81151178, 156.85391459, 152.92025838]), array([26.37890336, 32.9272513 , 39.56984817, 45.90784306, 51.40821563,
       55.36958427, 57.04189708, 56.01643288, 52.55145648, 47.35751724,
       41.16909338, 34.5601147 , 27.95680885, 2

In [ ]:
#Brute force way to stop current program_track

#any length 1 to 50 will do, 0 results in error 
sim.load_program_track('LOAD_RESET', 1)
sim.wait_duration(3)

In [ ]:
'''
States needed on SCU:
1. Command authority
2. DMC Activate 
A track table mode?
'''

#Get command authority
sim.command_authority('Get')
sim.wait_duration(5)

#Activate drives to be in SIP mode?
sim.activate_dmc();
sim.wait_duration(15)

'''
Load program track can be called with three load types:
LOAD_NEW
LOAD_ADD
LOAD_RESET
'''
#any length 1 to 50 will do, 0 results in error 
sim.load_program_track('LOAD_RESET', 1)
sim.wait_duration(3)

In [ ]:
#sim.deactivate_dmc()
#sim.reset_dmc()
sim.activate_dmc()